In [ ]:
from calendar import calendar
from selenium import webdriver
import time
from seleniumwire import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from seleniumwire.utils import decode
import json

In [ ]:
data_json = 'data.json'

def get_data(filename=data_json):
  with open(filename) as json_file:
      data = json.load(json_file)
  return data

def set_data(data, filename=data_json):
  data = json.dumps(data, indent=2, ensure_ascii=False)
  with open(filename, 'w') as f:
    f.write(data)

In [ ]:
def decode_body(response):
    result = decode(response.body, response.headers.get('Content-Encoding', 'identity'))
    # result = result.decode('unicode_escape')
    try:
        result_json = json.loads(result)
        return result_json
    except:
        return result

In [ ]:
url = 'https://sh-open.ris61edu.ru/desk'

path2chrome = './chromedriver'

chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=chrome_options, executable_path = path2chrome)

In [ ]:
try:
    driver.get(url=url)
except Exception as ex:
    print(f'Selenium exeption:\n{ex}\n')

## Reload page and get data

In [ ]:
driver.find_element_by_class_name("x-tbar-loading").click()
time.sleep(5)

### Rows

In [ ]:
rows = '{}'
for i in range(20):
    request = driver.requests[-i]
    if "registerrowsaction" in request.url:
        rows = request
print(rows)
rows = decode_body(rows.response)['rows']
print(len(rows))
print(rows)

In [ ]:
for row in rows:
    row['name'] = row['full_name'].split('>')[1].split('<')[0]

In [ ]:
set_data(rows, '../rows.json')

### Columns

In [ ]:
columns_request = {}
for i in range(20):
    request = driver.requests[-i]
    if "registercreategridaction" in request.url:
        columns_request = request
print(columns_request)

In [ ]:
columns = decode_body(columns_request.response)
columns = str(columns).split('Ext.data.Record.create(')[1].split(')')[0]
columns = [column[10:-1] for column in columns.split(',') if '{"name": "mix' in column or '"name": "pmt' in column]
print(columns)
len(columns)

In [ ]:
columns_dates = decode_body(columns_request.response)
columns_dates = str(columns_dates).split('LockingGridColumnWithHeaderGroup(')[1].split('})')[0]
columns_dates = columns_dates.replace('\\\\', """\\""")
columns_dates = json.loads(columns_dates + '}')['columnModelCfg']['rows'][0]
columns_dates
columns_dates = [column['header'].split('value="')[-1].split('"')[0] for column in columns_dates if column['header'] != '']
columns_dates = [column.replace('&nbsp;', ' - ') for column in columns_dates]
print(len(columns_dates))
print(columns_dates)

In [ ]:
columns_data = []
for index, title in enumerate(columns_dates):
    columns_data.append({'title': title, 'value': columns[index]})
set_data(columns_data, '../columns.json')

### Put mark request data
needs to put mark previously

In [ ]:
put_mark_request = {}
for i in range(20):
    request = driver.requests[-i]
    if "registerrowsaction" in request.url:
        put_mark_request = request
print(put_mark_request)

In [ ]:
put_mark_request_body = put_mark_request.body.decode('unicode_escape').split('&rows')[0]
put_mark_request_body += '&rows=*params*&xaction=update'
put_mark_request_body

In [ ]:
cookie = put_mark_request.headers['Cookie']
cookie

In [ ]:
set_data({'cookie': cookie, 'body': put_mark_request_body}, '../put_mark.json')

## Close selenium

In [ ]:
# driver.close()
# driver.quit()